<a href="https://colab.research.google.com/github/aslankims/Diabetes_Raman/blob/main/raman_spectroscopy_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Guevara, E., Torres-Galván, J. C., Ramírez-Elías, M. G., Luevano-Contreras, C., & González, F. J. (2018). Use of Raman spectroscopy to screen diabetes mellitus with machine learning tools. Biomedical Optics Express, 9(10), 4998–5010. https://doi.org/10.1364/BOE.9.004998


https://www.kaggle.com/sebastianlangan/raman-spectroscopy-project/data

Let's start with standard data imports, alloting each one of the five .csv files from the Raman Spectroscopy study its own Pandas DataFrame. 

In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

In [14]:
RamanAGEsDF = pd.read_csv("https://raw.githubusercontent.com/aslankims/Diabetes_Raman/main/AGEs.csv")
RamanInnerArmDF = pd.read_csv("https://raw.githubusercontent.com/aslankims/Diabetes_Raman/main/innerArm.csv")
RamanThumbnailDF = pd.read_csv("https://raw.githubusercontent.com/aslankims/Diabetes_Raman/main//thumbNail.csv")
RamanEarlobeDF = pd.read_csv("https://raw.githubusercontent.com/aslankims/Diabetes_Raman/main/earLobe.csv")
RamanVeinDF = pd.read_csv("https://raw.githubusercontent.com/aslankims/Diabetes_Raman/main/vein.csv")

In [15]:
#Let's take a quick peek at some of the Inner Arm spectral data, and make a quick plot of it: 
RamanInnerArmDF.head()

,patientID,has_DM2,Var2,Var3,Var4,Var5,Var6,Var7,Var8,Var9,Var10,Var11,Var12,Var13,Var14,Var15,Var16,Var17,Var18,Var19,Var20,Var21,Var22,Var23,Var24,Var25,Var26,Var27,Var28,Var29,Var30,Var31,Var32,Var33,Var34,Var35,Var36,Var37,Var38,Var39,...,Var3122,Var3123,Var3124,Var3125,Var3126,Var3127,Var3128,Var3129,Var3130,Var3131,Var3132,Var3133,Var3134,Var3135,Var3136,Var3137,Var3138,Var3139,Var3140,Var3141,Var3142,Var3143,Var3144,Var3145,Var3146,Var3147,Var3148,Var3149,Var3150,Var3151,Var3152,Var3153,Var3154,Var3155,Var3156,Var3157,Var3158,Var3159,Var3160,Var3161
0,ramanShift,NaN,0.000000,1.000000,2.000000,3.000000,4.000000,5.000000,6.000000,7.000000,8.000000,9.000000,10.000000,11.000000,12.000000,13.000000,14.000000,15.000000,16.000000,17.000000,18.000000,19.000000,20.000000,21.000000,22.000000,23.000000,24.000000,25.000000,26.000000,27.000000,28.000000,29.000000,30.000000,31.000000,32.000000,33.000000,34.000000,35.000000,36.000000,37.000000,...,3120,3121,3122,3123,3124,3125,3126,3127,3128,3129,3130,3131,3132,3133,3134,3135,3136,3137,3138,3139,3140,3141,3142,3143,3144,3145,3146,3147,3148,3149,3150,3151,3152,3153,3154,3155,3156,3157,3158,3159
1,DM201,1.0,326.000000,326.000000,326.000000,326.000000,326.000000,326.000000,326.000000,326.000000,326.000000,326.000000,326.000000,326.000000,326.000000,326.000000,326.000000,326.000000,326.000000,326.000000,326.000000,326.000000,326.000000,326.000000,326.000000,326.000000,326.000000,326.000000,326.000000,326.000000,326.000000,326.000000,326.000000,326.000000,326.000000,326.000000,326.000000,326.000000,326.000000,326.000000,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,DM202,1.0,214.800000,214.800000,214.800000,214.800000,214.800000,214.800000,214.800000,214.800000,214.800000,214.800000,214.800000,214.800000,214.800000,214.800000,214.800000,214.800000,214.800000,214.800000,214.800000,214.800000,214.800000,214.800000,214.800000,214.800000,214.800000,214.800000,214.800000,214.800000,214.800000,214.800000,214.800000,214.800000,214.800000,214.800000,214.800000,214.800000,214.800000,214.800000,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,DM203,1.0,457.800000,457.800000,457.800000,457.800000,457.800000,457.800000,457.800000,457.800000,457.800000,457.800000,457.800000,457.800000,457.800000,457.800000,457.800000,457.800000,457.800000,457.800000,457.800000,457.800000,457.800000,457.800000,457.800000,457.800000,457.800000,457.800000,457.800000,457.800000,457.800000,457.800000,457.800000,457.800000,457.800000,457.800000,457.800000,457.800000,457.800000,457.800000,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,DM204,1.0,181.833333,181.833333,181.833333,181.833333,181.833333,181.833333,181.833333,181.833333,181.833333,181.833333,181.833333,181.833333,181.833333,181.833333,181.833333,181.833333,181.833333,181.833333,181.833333,181.833333,181.833333,181.833333,181.833333,181.833333,181.833333,181.833333,181.833333,181.833333,181.833333,181.833333,181.833333,181.833333,181.833333,181.833333,181.833333,181.833333,181.833333,181.833333,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [16]:
#Let's select the first row of the Inner Arm DataFrame to be used as the domain in future plots or analyses: 
InnerArmWavenumbersDF = RamanInnerArmDF.iloc[0]

#Let's also drop the first row (containing the Raman wavenumbers, the spectral domain) from the original DataFrame: 
RamanInnerArmDF = RamanInnerArmDF.drop(RamanInnerArmDF.index[0])
RamanInnerArmDF = RamanInnerArmDF.reset_index(drop=True)

In [17]:
RamanInnerArmDF.head()

,patientID,has_DM2,Var2,Var3,Var4,Var5,Var6,Var7,Var8,Var9,Var10,Var11,Var12,Var13,Var14,Var15,Var16,Var17,Var18,Var19,Var20,Var21,Var22,Var23,Var24,Var25,Var26,Var27,Var28,Var29,Var30,Var31,Var32,Var33,Var34,Var35,Var36,Var37,Var38,Var39,...,Var3122,Var3123,Var3124,Var3125,Var3126,Var3127,Var3128,Var3129,Var3130,Var3131,Var3132,Var3133,Var3134,Var3135,Var3136,Var3137,Var3138,Var3139,Var3140,Var3141,Var3142,Var3143,Var3144,Var3145,Var3146,Var3147,Var3148,Var3149,Var3150,Var3151,Var3152,Var3153,Var3154,Var3155,Var3156,Var3157,Var3158,Var3159,Var3160,Var3161
0,DM201,1.0,326.000000,326.000000,326.000000,326.000000,326.000000,326.000000,326.000000,326.000000,326.000000,326.000000,326.000000,326.000000,326.000000,326.000000,326.000000,326.000000,326.000000,326.000000,326.000000,326.000000,326.000000,326.000000,326.000000,326.000000,326.000000,326.000000,326.000000,326.000000,326.000000,326.000000,326.000000,326.000000,326.000000,326.000000,326.000000,326.000000,326.000000,326.000000,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,DM202,1.0,214.800000,214.800000,214.800000,214.800000,214.800000,214.800000,214.800000,214.800000,214.800000,214.800000,214.800000,214.800000,214.800000,214.800000,214.800000,214.800000,214.800000,214.800000,214.800000,214.800000,214.800000,214.800000,214.800000,214.800000,214.800000,214.800000,214.800000,214.800000,214.800000,214.800000,214.800000,214.800000,214.800000,214.800000,214.800000,214.800000,214.800000,214.800000,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,DM203,1.0,457.800000,457.800000,457.800000,457.800000,457.800000,457.800000,457.800000,457.800000,457.800000,457.800000,457.800000,457.800000,457.800000,457.800000,457.800000,457.800000,457.800000,457.800000,457.800000,457.800000,457.800000,457.800000,457.800000,457.800000,457.800000,457.800000,457.800000,457.800000,457.800000,457.800000,457.800000,457.800000,457.800000,457.800000,457.800000,457.800000,457.800000,457.800000,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,DM204,1.0,181.833333,181.833333,181.833333,181.833333,181.833333,181.833333,181.833333,181.833333,181.833333,181.833333,181.833333,181.833333,181.833333,181.833333,181.833333,181.833333,181.833333,181.833333,181.833333,181.833333,181.833333,181.833333,181.833333,181.833333,181.833333,181.833333,181.833333,181.833333,181.833333,181.833333,181.833333,181.833333,181.833333,181.833333,181.833333,181.833333,181.833333,181.833333,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,DM205,1.0,179.800000,179.800000,179.800000,179.800000,179.800000,179.800000,179.800000,179.800000,179.800000,179.800000,179.800000,179.800000,179.800000,179.800000,179.800000,179.800000,179.800000,179.800000,179.800000,179.800000,179.800000,179.800000,179.800000,179.800000,179.800000,179.800000,179.800000,179.800000,179.800000,179.800000,179.800000,179.800000,179.800000,179.800000,179.800000,179.800000,179.800000,179.800000,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [18]:
RamanInnerArmDF.tail()

,patientID,has_DM2,Var2,Var3,Var4,Var5,Var6,Var7,Var8,Var9,Var10,Var11,Var12,Var13,Var14,Var15,Var16,Var17,Var18,Var19,Var20,Var21,Var22,Var23,Var24,Var25,Var26,Var27,Var28,Var29,Var30,Var31,Var32,Var33,Var34,Var35,Var36,Var37,Var38,Var39,...,Var3122,Var3123,Var3124,Var3125,Var3126,Var3127,Var3128,Var3129,Var3130,Var3131,Var3132,Var3133,Var3134,Var3135,Var3136,Var3137,Var3138,Var3139,Var3140,Var3141,Var3142,Var3143,Var3144,Var3145,Var3146,Var3147,Var3148,Var3149,Var3150,Var3151,Var3152,Var3153,Var3154,Var3155,Var3156,Var3157,Var3158,Var3159,Var3160,Var3161
15,Ctrl05,0.0,71.833333,71.833333,71.833333,71.833333,71.833333,71.833333,71.833333,71.833333,71.833333,71.833333,71.833333,71.833333,71.833333,71.833333,71.833333,71.833333,71.833333,71.833333,71.833333,71.833333,71.833333,71.833333,71.833333,71.833333,71.833333,71.833333,71.833333,71.833333,71.833333,71.833333,71.833333,71.833333,71.833333,71.833333,71.833333,71.833333,71.833333,71.833333,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
16,Ctrl06,0.0,325.600000,325.600000,325.600000,325.600000,325.600000,325.600000,325.600000,325.600000,325.600000,325.600000,325.600000,325.600000,325.600000,325.600000,325.600000,325.600000,325.600000,325.600000,325.600000,325.600000,325.600000,325.600000,325.600000,325.600000,325.600000,325.600000,325.600000,325.600000,325.600000,325.600000,325.600000,325.600000,325.600000,325.600000,325.600000,325.600000,325.600000,325.600000,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
17,Ctrl07,0.0,116.000000,116.000000,116.000000,116.000000,116.000000,116.000000,116.000000,116.000000,116.000000,116.000000,116.000000,116.000000,116.000000,116.000000,116.000000,116.000000,116.000000,116.000000,116.000000,116.000000,116.000000,116.000000,116.000000,116.000000,116.000000,116.000000,116.000000,116.000000,116.000000,116.000000,116.000000,116.000000,116.000000,116.000000,116.000000,116.000000,116.000000,116.000000,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
18,Ctrl08,0.0,272.500000,272.500000,272.500000,272.500000,272.500000,272.500000,272.500000,272.500000,272.500000,272.500000,272.500000,272.500000,272.500000,272.500000,272.500000,272.500000,272.500000,272.500000,272.500000,272.500000,272.500000,272.500000,272.500000,272.500000,272.500000,272.500000,272.500000,272.500000,272.500000,272.500000,272.500000,272.500000,272.500000,272.500000,272.500000,272.500000,272.500000,272.500000,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
19,Ctrl09,0.0,156.166667,156.166667,156.166667,156.166667,156.166667,156.166667,156.166667,156.166667,156.166667,156.166667,156.166667,156.166667,156.166667,156.166667,156.166667,156.166667,156.166667,156.166667,156.166667,156.166667,156.166667,156.166667,156.166667,156.166667,156.166667,156.166667,156.166667,156.166667,156.166667,156.166667,156.166667,156.166667,156.166667,156.166667,156.166667,156.166667,156.166667,156.166667,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


1) Now, starting with the Inner Arm Dataframe, I'll normalize and run PCA on the data to reduce its dimensionality down to 15. The paper associated with this dataset mentioned that to be the optimal number of components for this chunk of the data. 

In [19]:
#1a) Inner Arm Data Scaling: 
from sklearn.preprocessing import StandardScaler 

#I'm making a slice of the original dataframe, from the "Var2" column onwards, as each column is an independent feature representing a wavenumber 
#chunk of the spectral domain. I'll then make a list of this dataframe of column names. This will be an input in the StandardScaler functions:
innerArmFeaturesDataFrame = RamanInnerArmDF.iloc[:,2:3159]
innerArmFeaturesList = innerArmFeaturesDataFrame.columns.tolist()

innerArmFeaturesValues = innerArmFeaturesDataFrame.loc[:,innerArmFeaturesList].values

#Separating out target values: 
innerArmTargetValues = RamanInnerArmDF.loc[:,['has_DM2']].values

#Feature Standardization: 
RamanInnerArmStandardScalerObject = StandardScaler().fit_transform(innerArmFeaturesValues)

In [20]:
#1b) Inner Arm PCA, set to (the reccomended) 15 components: 
from sklearn.decomposition import PCA

RamanInnerArmPCA = PCA(n_components = 15) 
RamanInnerArmPrincipalComponents = RamanInnerArmPCA.fit_transform(RamanInnerArmStandardScalerObject)

RamanInnerArmPCADF = pd.DataFrame(data = RamanInnerArmPrincipalComponents
             , columns = ['principal component 1', 'principal component 2', 'principal component 3', 'principal component 4', 'principal component 5', 'principal component 6', 'principal component 7', 'principal component 8', 'principal component 9', 'principal component 10', 'principal component 11', 'principal component 12', 'principal component 13', 'principal component 14', 'principal component 15'])

#Now that we have a total of fifteen new principal components, preserving as much variation from the original 3159 Raman
#wavenumber features, lets concatenate the new primary components DF with the features column in a new DataFrame:
RamanPCAInnerArmComponentsAndFeatureDF = pd.concat([RamanInnerArmPCADF, RamanInnerArmDF[['has_DM2']]], axis = 1)


In [21]:
#Let's get a little preview of that concatenated DataFrame: 
RamanPCAInnerArmComponentsAndFeatureDF.head(20)

,principal component 1,principal component 2,principal component 3,principal component 4,principal component 5,principal component 6,principal component 7,principal component 8,principal component 9,principal component 10,principal component 11,principal component 12,principal component 13,principal component 14,principal component 15,has_DM2
0,47.222918,7.145680,-6.683618,3.167498,-2.925279,-3.181392,-0.646655,-0.490315,0.517423,2.732065,-1.798149,-2.731270,1.666045,-2.175124,4.387751,1.0
1,-3.859357,-6.496048,-2.691251,-2.147266,-2.785565,-2.488168,-1.655383,-0.878841,1.313479,-3.785902,-1.223921,2.204281,2.714711,4.713958,1.396491,1.0
2,114.666707,16.222721,-3.342684,3.011577,-0.047054,-0.047162,-2.521048,-0.116957,1.470904,-2.166610,-1.419890,3.451132,-0.782089,-2.448261,-2.878530,1.0
3,-23.901991,-6.559510,-1.969945,-0.001370,3.113465,-0.931984,-0.225603,-0.674807,0.844730,-0.809939,-0.125858,-0.721037,-0.525770,2.061907,1.210663,1.0
4,-28.766550,2.168372,-4.174635,3.541340,3.780820,-1.356589,-1.929865,1.107052,1.978740,1.490739,-1.798523,-1.289930,-3.448901,1.933798,-0.787482,1.0
5,-22.661784,43.804084,2.117221,-1.983106,-1.357085,1.416356,0.552067,0.143583,-0.875454,-0.657263,0.527516,0.465433,0.503147,0.094374,0.435004,1.0
6,-3.418142,-5.073555,-0.169552,-0.574871,3.439057,-1.915368,0.736279,-0.059795,4.945420,0.620146,4.031459,-1.585228,4.058599,-1.207408,-2.327849,1.0
7,-28.671182,-13.595802,9.004169,4.693889,-3.205121,0.119547,-0.300286,-2.364773,-1.286036,-1.531081,-3.770842,-3.121794,1.676568,-0.584971,-2.698592,1.0
8,64.647891,-2.988138,7.785108,-4.176278,4.345674,-0.053066,2.884967,-3.592821,0.500360,4.266789,-2.250115,2.425795,0.127000,0.860364,0.745772,1.0
9,-70.516281,-11.785179,-4.472854,-0.404690,3.016580,3.629371,-0.433539,-0.458165,1.326049,0.319260,-1.538527,0.433346,-1.724034,-0.935326,-0.582094,1.0


2) Now that the Inner Arm Data has been normalized and reduced, let's implement a K-Fold cross-validation method and attempt to 
replicate the CV procedures described in the original paper for this dataset. Since the paper mentioned that their team repeated K-fold 
CV and their subsequent SVM/ANN runs one thousand times, I will attempt to repeat a similar process with CV --> Binomial Logistic Regression.

In [ ]:
#2) Implementing K-Fold cross-validation and Logistic Regression, iterating one thousand times: 

#Necessary imports: 
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score 

#Splitting the PCA-reduced Dataframe into Target and Feature Data: 
RamanPCAInnerArmFeaturesDF = RamanPCAInnerArmComponentsAndFeatureDF.drop(RamanPCAInnerArmComponentsAndFeatureDF[['has_DM2']], axis = 1)
RamanPCAInnerArmTargetDF = RamanPCAInnerArmComponentsAndFeatureDF.loc[:,'has_DM2']
#I'm going to drop the first rows of both Dataframes because the target value for the first partipant now registers as "NaN" for some 
#reason:
RamanPCAInnerArmFeaturesDF = RamanPCAInnerArmFeaturesDF.drop(RamanPCAInnerArmFeaturesDF.index[0])
RamanPCAInnerArmTargetDF = RamanPCAInnerArmTargetDF.drop(RamanPCAInnerArmTargetDF.index[0])

#Creating empty lists to store test accuracy results: 
avgLRScore = []
LRScoresCV5 = []

#**I keep receiving an error saying that cross_val_score cannot run correctly because there is either a NaN, infinite, or too large 
#value. I'm going to convert the Dataframes into numpy arrays to see if this is a compatibility issue**#
RamanPCAInnerArmFeaturesArray = np.asarray(RamanPCAInnerArmFeaturesDF)
RamanPCAInnerArmTargetArray = np.asarray(RamanPCAInnerArmTargetDF)

RamanPCAInnerArmComponentsAndFeatureDF.head(25)

#Outer for loop, to iterate a 1000 times. Will do this over a list over a certain integer range (1-1000)#
for i in range(1,1000):
    LRScoresCV5.append(cross_val_score(LogisticRegression(),RamanPCAInnerArmFeaturesArray,RamanPCAInnerArmTargetArray, cv = 5))

avgLRScore = np.average(LRScoresCV5)

In [23]:
avgLRScore

0.5833333333333334

It doesn't seem like regular, two-class Logistic Regression is any more effective than just guessing by pure chance.